In [1]:
# MNIST 데이터셋으로 분류기를 만들어 테스트 세트에서 97% 정확도를 달성해보세요.

In [1]:
# 라이브러리 import

import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import accuracy_score

In [2]:
# mnist 데이터셋 획득

def get_mnist():
    mnist = fetch_openml('mnist_784', version=1, as_frame=False)
    X, y = mnist["data"], mnist["target"]
    y = y.astype(np.uint8)
    X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]
    return X_train, X_test, y_train, y_test

In [3]:
X_train, X_test, y_train, y_test = get_mnist()

C:\Users\anfcl\anaconda3\Lib\site-packages\sklearn\datasets\_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [4]:
# X data standardization or Normalization

class standard_or_normal(BaseEstimator):
    def __init__(self, select_standard = True):
        self.select_standard = select_standard
        self.scaler = StandardScaler()
        self.normalizer = MinMaxScaler()
        
    def fit(self, X, y=None):
        if self.select_standard:
            self.scaler.fit(X)
        else:
            self.normalizer.fit(X)
        return self
    
    def transform(self, X):
        if self.select_standard:
            return self.scaler.transform(X)
        else:
            return self.normalizer.transform(X)

In [5]:
# Pipeline 생성

pipeline = Pipeline([
    ('standard_or_normal', standard_or_normal(select_standard=True)),
    ('knn_clf', KNeighborsClassifier())
])

In [6]:
# 최적의 KNeighborsClassifier 랜덤 탐색

param = {
    'standard_or_normal__select_standard':[True, False],
    'knn_clf__n_neighbors':range(1, 11),
    'knn_clf__weights':['uniform', 'distance']
}

rs = GridSearchCV(pipeline, param_grid=param, cv=3, verbose=3)
rs.fit(X_train, y_train)

Fitting 3 folds for each of 40 candidates, totalling 120 fits
[CV 1/3] END knn_clf__n_neighbors=1, knn_clf__weights=uniform, standard_or_normal__select_standard=True;, score=0.935 total time=   3.3s
[CV 2/3] END knn_clf__n_neighbors=1, knn_clf__weights=uniform, standard_or_normal__select_standard=True;, score=0.939 total time=   2.9s
[CV 3/3] END knn_clf__n_neighbors=1, knn_clf__weights=uniform, standard_or_normal__select_standard=True;, score=0.941 total time=   2.8s
[CV 1/3] END knn_clf__n_neighbors=1, knn_clf__weights=uniform, standard_or_normal__select_standard=False;, score=0.969 total time=   2.8s
[CV 2/3] END knn_clf__n_neighbors=1, knn_clf__weights=uniform, standard_or_normal__select_standard=False;, score=0.967 total time=   2.7s
[CV 3/3] END knn_clf__n_neighbors=1, knn_clf__weights=uniform, standard_or_normal__select_standard=False;, score=0.967 total time=   2.8s
[CV 1/3] END knn_clf__n_neighbors=1, knn_clf__weights=distance, standard_or_normal__select_standard=True;, score=

[CV 3/3] END knn_clf__n_neighbors=5, knn_clf__weights=distance, standard_or_normal__select_standard=False;, score=0.969 total time=   2.8s
[CV 1/3] END knn_clf__n_neighbors=6, knn_clf__weights=uniform, standard_or_normal__select_standard=True;, score=0.936 total time=   2.9s
[CV 2/3] END knn_clf__n_neighbors=6, knn_clf__weights=uniform, standard_or_normal__select_standard=True;, score=0.939 total time=   2.9s
[CV 3/3] END knn_clf__n_neighbors=6, knn_clf__weights=uniform, standard_or_normal__select_standard=True;, score=0.940 total time=   3.0s
[CV 1/3] END knn_clf__n_neighbors=6, knn_clf__weights=uniform, standard_or_normal__select_standard=False;, score=0.965 total time=   2.8s
[CV 2/3] END knn_clf__n_neighbors=6, knn_clf__weights=uniform, standard_or_normal__select_standard=False;, score=0.965 total time=   2.8s
[CV 3/3] END knn_clf__n_neighbors=6, knn_clf__weights=uniform, standard_or_normal__select_standard=False;, score=0.966 total time=   2.8s
[CV 1/3] END knn_clf__n_neighbors=6,

[CV 3/3] END knn_clf__n_neighbors=10, knn_clf__weights=distance, standard_or_normal__select_standard=False;, score=0.966 total time=   2.9s


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('standard_or_normal',
                                        standard_or_normal()),
                                       ('knn_clf', KNeighborsClassifier())]),
             param_grid={'knn_clf__n_neighbors': range(1, 11),
                         'knn_clf__weights': ['uniform', 'distance'],
                         'standard_or_normal__select_standard': [True, False]},
             verbose=3)

In [7]:
rs.best_params_

{'knn_clf__n_neighbors': 4,
 'knn_clf__weights': 'distance',
 'standard_or_normal__select_standard': False}

In [8]:
final_model = rs.best_estimator_

final_prediction = final_model.predict(X_test)

accuracy = accuracy_score(y_test, final_prediction)
print(accuracy)


0.9714
